# Ratio Pileups 
 <p>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.gridspec
import matplotlib.pyplot as plt
import cooler
import cooltools
from cooltools import snipping
import bioframe

%matplotlib inline

In [ ]:
plt.rcParams['pdf.fonttype'] = 'truetype'
plt.rcParams['svg.fonttype'] = 'none' # No text as paths. Assume font installed.

plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['text.usetex'] = False

In [ ]:
# add fall colormap

def listToColormap(colorList, cmapName=None):
    colorList = np.array(colorList)
    if colorList.min() < 0:
        raise ValueError("Colors should be 0 to 1, or 0 to 255")
    if colorList.max() > 1.:
        if colorList.max() > 255:
            raise ValueError("Colors should be 0 to 1 or 0 to 255")
        else:
            colorList = colorList / 255.
    return matplotlib.colors.LinearSegmentedColormap.from_list(cmapName, colorList, 256)

fallList = [
     (255, 255, 255), (255, 255, 204),
     (255, 237, 160), (254, 217, 118),
     (254, 178, 76), (253, 141, 60),
     (252, 78, 42), (227, 26, 28),
     (189, 0, 38), (128, 0, 38), (0, 0, 0)]


def registerList(mylist, name):
    mymap = listToColormap(mylist, name)
    mymapR = listToColormap(mylist[::-1], name + "_r")
    mymapR.set_bad('white',1.)
    mymap.set_bad('white',1.)
    matplotlib.cm.register_cmap(name, mymap)
    matplotlib.cm.register_cmap(name + "_r", mymapR)

registerList(fallList, "fall")

coolwarm = matplotlib.cm.coolwarm
coolwarm.set_bad('white',1.)
matplotlib.cm.register_cmap('coolwarm', coolwarm)

In [ ]:
chromsizes = pd.read_table(
    'chrom.sizes.file', 
    header=None,
    names=['chrom', 'length'])
chromsizes

In [ ]:
chromsizeslist = cooler.util.read_chromsizes(
    'chrom.sizes.file', 
    name_patterns=(r'^chr[IVX]+$', r'chrMito$'))
chromsizeslist

In [ ]:
chromosomes = list(chromsizeslist.index)
chromosomes

In [ ]:
# Load sites of interest
sites = pd.read_table('sites.bed.file')

sites.head()

In [ ]:
supports = [(chrom[0], 0, chrom[1])
            for _, chrom in chromsizes.iterrows()
           ]
supports

In [ ]:
supports_p = bioframe.parse_regions(supports)
supports_p

In [ ]:
sites['mid'] = (sites['start'] + sites['end']) / 2

binsize = 1000
flank = 25000

cens = snipping.make_bin_aligned_windows(
    binsize,
    sites.chrom.values,
    sites.mid.values,
    flank # set flank_bp here
)
cens['strand'] = sites['strand']
cens = snipping.assign_regions(cens, supports_p) 
cens = cens.dropna()
print(len(cens), 'cen windows, after assigning supports')

cens.head()

In [ ]:
cens['dummy'] = 1
cen_pairs = pd.merge(cens, cens, how='outer', on='dummy', suffixes=['1','2'])[
    ['chrom1', 'start1', 'end1', 'strand1', 'region1', 'chrom2', 'start2', 'end2', 'strand2','region2']]

cen_pairs['region'] = list(zip(cen_pairs['region1'].values,
                               cen_pairs['region2'].values))

cen_pairs.head()

In [ ]:
# All
chroms4pileup = [
    'chrI',
    'chrII',
    'chrIII',
    'chrIV',
    'chrV',
    'chrVI',
    'chrVII',
    'chrVIII',
    'chrIX',
    'chrX',
    'chrXI',
    'chrXII',
    'chrXIII',
    'chrXIV',
    'chrXV',
    'chrXVI']

cis_cen_pairs = cen_pairs[
    (cen_pairs.chrom1 == cen_pairs.chrom2) & (cen_pairs.strand1 == cen_pairs.strand2)
    & cen_pairs.chrom1.isin(chroms4pileup)
    & cen_pairs.chrom2.isin(chroms4pileup)
]
cis_cen_pairs.reset_index(drop=True, inplace=True)

cis_cen_pairs.head()

In [ ]:
cis_cen_pileup_dict = {}

import pathlib
for p in pathlib.Path('PATH_TO_COOL').glob('*[0-9].cool'):
    sample = p.name.split('.')[0]
    print(sample, '...')
    c = cooler.Cooler(p.as_posix())
    snipper = snipping.CoolerSnipper(c, regions=supports_p)

    cis_cen_pile = snipping.pileup( 
        cis_cen_pairs, 
        snipper.select, 
        snipper.snip) 
      
    # mirror reflect snippets whose feature is on the opposite strand
    mask = np.array(cens.strand == '-', dtype=bool)
    cis_cen_pile[:, :, mask] =  cis_cen_pile[::-1, ::-1, mask]
    
    cis_cen_pileup_dict[sample] = np.nanmean(cis_cen_pile, axis=2)

In [ ]:
shortnames = {
     'long_name_A' : 'A',
     'long_name_B' : 'B'
}

def plot_pileup_ratios(pileup_dict, use_log2=False, hm_kwargs={}):
    
    vmin=hm_kwargs.get('vmin',-1.5)
    vmax=hm_kwargs.get('vmax',1.5)
    for i, dataset_pair in enumerate([
        ('A', 'B')
        ]):

        if dataset_pair[0] is None:
            continue
        
        ratio_hm = pileup_dict[dataset_pair[0]]/pileup_dict[dataset_pair[1]]
        print('max log2', np.log2(np.max(ratio_hm[np.isfinite(ratio_hm)])),
              'min log2', np.log2(np.min(ratio_hm[np.isfinite(ratio_hm)])))
        log_ratio_hm = np.log2(ratio_hm) if use_log2 else np.log10(ratio_hm)
        
        plt.figure(figsize=(8, 8))

        heatplot = plt.imshow(
            log_ratio_hm,
            extent=[-flank//1000, flank//1000, -flank//1000, flank//1000],
            cmap=plt.cm.get_cmap('coolwarm'),
            interpolation='none',
            vmin=vmin,
            vmax=vmax)

        plt.title('{} / {}'.format(shortnames[dataset_pair[0]],shortnames[dataset_pair[1]]))  
       
        cb = plt.colorbar(heatplot, fraction=0.1,  ticks=[vmin, 0, vmax]) 
        plt.grid(False)
        plt.savefig('{}.{}.flank{}kb.bin{}kb.log2.Ratio_Pileup.png'.format(shortnames[dataset_pair[0]],shortnames[dataset_pair[1]], flank//1000, binsize//1000))

    cb.set_label(('log2' if use_log2 else 'log10') + ' ratio')


In [ ]:
plot_pileup_ratios(
    cis_cen_pileup_dict,
    use_log2=True,
    hm_kwargs = dict(vmin=-1.5, vmax=1.5))